<a href="https://colab.research.google.com/github/williamlidberg/Unet-tutorial/blob/main/U_net_tutorial_on_impact_craters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to google colab

# Cells
A notebook is a list of cells. Cells contain either explanatory text or executable code and its output. Click a cell to select it.

## Code cells
Below is a **code cell**. Once the toolbar button indicates CONNECTED, click in the cell to select it and execute the contents in the following ways:

* Click the **Play icon** in the left gutter of the cell;
* Type **Cmd/Ctrl+Enter** to run the cell in place;
* Type **Shift+Enter** to run the cell and move focus to the next cell (adding one if none exists); or
* Type **Alt+Enter** to run the cell and insert a new code cell immediately below it.

There are additional options for running some or all cells in the **Runtime** menu.


In [ ]:
a = 10
a

# Working with python
Colaboratory is built on top of [Jupyter Notebook](https://jupyter.org/). Below are some examples of convenience functions provided.

In [ ]:
import time
print("Sleeping")
time.sleep(30) # sleep for a while; interrupt me!
print("Done Sleeping")

## Adding and moving cells
You can add new cells by using the **+ CODE** and **+ TEXT** buttons that show when you hover between cells. These buttons are also in the toolbar above the notebook where they can be used to add a cell below the currently selected cell.

You can move a cell by selecting it and clicking **Cell Up** or **Cell Down** in the top toolbar. 

Consecutive cells can be selected by "lasso selection" by dragging from outside one cell and through the group.  Non-adjacent cells can be selected concurrently by clicking one and then holding down Ctrl while clicking another.  Similarly, using Shift instead of Ctrl will select all intermediate cells.

# Deep learning on the moon
In this tutorial you will learn:


1.   Google colab
2.   Import python libraries
3.   Run python code
4.   Import data and plot images
5.   Build a deep learning pipline
6.   Build a basic Unet model
7.   Train and test a Unet model
8.   Adjust weights to classes


## Import libraries
Python libraries are like R packages. Before they can be used they need to be imported into the python environment.

In [3]:
import os # For data handling
import glob # For data handling
import imageio # For images
import cv2 # For images
import numpy as np # For arrays
import tensorflow.keras.backend as K # For deep learning
import tensorflow as tf # For deep learning
from skimage.transform import resize # For preprocessing
import matplotlib.pyplot as plt # For plotting data
from skimage.io import imread, imshow # for plots
import random # for seeds
random.seed(10) # To increase reproducability
tf.__version__

'2.8.2'

## Import data

In [ ]:
%%time
!git clone https://github.com/williamlidberg/Unet-tutorial.git

## Inspect data
The first stepp in any data analysis is always to inspect the data. The original lunar craters sort of look like hunting pits while the inverted data looks more like charcoal kilns. Start by checking the number of labeled files in the training data and testing data.

In [ ]:
labels = glob.glob('/content/Unet-tutorial/craters_1000_samples/train/labels/*.png')
print(len(labels),'images in the training data')

labels = glob.glob('/content/Unet-tutorial/craters_1000_samples/test/labels/*.png')
print(len(labels),'in the test data')

Plot some samples of the images and labels. Note that the massive craters are not annotaded in this dataset.

In [ ]:
# size of plot
plt.rcParams['figure.figsize'] = [16, 16]

# make lists of images
labels = glob.glob('/content/Unet-tutorial/craters_1000_samples/train/labels/*.png')
labels.sort()
images = glob.glob('/content/Unet-tutorial/craters_1000_samples/test/images/*.png')
images.sort()

# select and plot images
f, axarr = plt.subplots(2,2)
axarr[0,0].set_title('Original data looks like hunting pits')
axarr[0,0].imshow(imageio.imread(images[0]), cmap='Greys_r')
axarr[1,0].set_title('Inverted data looks like charcoal kilns') 
axarr[1,0].imshow(imageio.imread(images[0]),cmap='Greys')

axarr[0,1].set_title('Lunar Craters')
axarr[0,1].imshow(imageio.imread(labels[0]))
axarr[1,1].set_title('Lunar Craters')
axarr[1,1].imshow(imageio.imread(labels[0]))



## Create training and testing data
The images and labels are stored as images in the directories train and test. This script loads each image as a numpy array into a list and then converts this list of arrays into a [Tensorflow dataset]( https://www.tensorflow.org/api_docs/python/tf/data/Dataset).


Each image is a 512x512 greyscale chips with only 1 band. (Normal color images have 3 bands, red, green and blue). The labels are binary where the value 0 indicates the background and 1 indicates a impact crater. The deep learning model expects values to range between 0 and 1 so the greyscale image will be normalized by deviding with the maximum value (255).

To save some time all images will be resized to 256x256 for this tutorial.

The "%%time" part is used to calculate how long time this script takes to run. This should take about 5 - 6 min.

In [ ]:
%%time
IMG_WIDTH = 256 # nr pixels
IMG_HEIGHT = 256
IMG_CHANNELS = 1 # a grey scale image only has one band for color.
NUM_CLASSES = 1 # 0 = no crater and 1 = crater

#### Training data ####
TRAIN_PATH = '/content/Unet-tutorial/craters_1000_samples/train/'
IMG_DIR = 'images'
GT_DIR = 'labels'
X_train = []
Y_train = []

# load from disk
img_path = os.path.join(TRAIN_PATH, IMG_DIR)
gt_path = os.path.join(TRAIN_PATH, GT_DIR)
for image in (os.listdir(img_path)[0:300]):
    img = imageio.imread(os.path.join(img_path, image))
    img = img /255
    img = resize(img, (IMG_WIDTH, IMG_HEIGHT,1), mode='constant', preserve_range=True)
    
    mask = imageio.imread(os.path.join(gt_path, image))
    mask = resize(mask, (IMG_WIDTH, IMG_HEIGHT, 1), preserve_range=True, order=0).astype(int)
    
    X_train.append(img)
    Y_train.append(mask)

#### Test data ####
TEST_PATH = '/content/Unet-tutorial/craters_1000_samples/test/'
IMG_DIR = 'images'
GT_DIR = 'labels'
X_test = []
Y_test = []

# load from disk
img_path = os.path.join(TEST_PATH, IMG_DIR)
gt_path = os.path.join(TEST_PATH, GT_DIR)
for image in (os.listdir(img_path)[0:300]):
    img = imageio.imread(os.path.join(img_path, image))
    img = img /255
    img = resize(img, (IMG_WIDTH, IMG_HEIGHT,1), mode='constant', preserve_range=True)
    
    mask = imageio.imread(os.path.join(gt_path, image))
    mask = resize(mask, (IMG_WIDTH, IMG_HEIGHT, 1), preserve_range=True, order=0).astype(int)
    
    X_test.append(img)
    Y_test.append(mask)

# convert list of numpy arrays into tensorflow dataset for further processing
train_images = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
test_images = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

## Build input pipelines
[Batch size](https://medium.com/deep-learning-experiments/effect-of-batch-size-on-neural-net-training-c5ae8516e57) is a term used in machine learning and refers to the number of training examples utilized in one iteration.
Buffer size is how many images that should be read into memory at the time. For a large number of images it is not possible to read the entire dataset into the system memory.

**Training pipeline**

In [8]:
BATCH_SIZE = 4
BUFFER_SIZE = 100
train_batches = (train_images 
                    .cache() # cache data
                    .shuffle(BUFFER_SIZE) # fill buffer, sample from it and replace with new items (buffer size > training set for perfect shuffling)
                    .batch(BATCH_SIZE)  # set batch size
                    .repeat()  # repeat dataset idefinetely
                    .prefetch(buffer_size=100)) # prefetch 100 images to optimize runtime

**Testing pipeline**

In [9]:
# setup input pipeline
BATCH_SIZE = 4
BUFFER_SIZE = 100
test_batches = (test_images 
                    .cache() # cache data
                    .shuffle(BUFFER_SIZE) # fill buffer, sample from it and replace with new items (buffer size > training set for perfect shuffling)
                    .batch(BATCH_SIZE)  # set batch size
                    .repeat(1)  # repeat dataset idefinetely
                    .prefetch(buffer_size=100)) # prefetch 100 images to optimize runtime

## Define metrics to use for evaluation
Accuracy is how many of the pixels are classified correctly by the model. This is not very usefull if most of the landscape consists of one class. In this case most of the lunar surface is infact not an impact crater. 

f1 score is a better option since it tries to take the inbalance of the data into account. f1 = 0 is very low and f1 = 1 is a perfect fit. Both recall and precision are required to calculate F1 score so these have to be defined as well.



In [10]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Build U-net model
[U-net](https://arxiv.org/abs/1505.04597) is a convolutional neural network that was developed for biomedical image segmentation. The network consists of a contracting path and an expansive path, which gives it the [u-shaped architecture](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/). The contracting path is a typical convolutional network that consists of repeated application of convolutions, each followed by a rectified linear unit (ReLU) and a max pooling operation. During the contraction, the spatial information is reduced while feature information is increased. The expansive pathway combines the feature and spatial information through a sequence of up-convolutions and concatenations with high-resolution features from the contracting path.

In [11]:
#Build the model
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
NUM_CLASSES = 1
#Contraction path
c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1) # to prevent overfitting
c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(NUM_CLASSES, (1, 1))(c9)

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryFocalCrossentropy(gamma=2.0, from_logits=True), metrics=['acc',f1_m, recall_m])



We can also print out a text summary of the model. Note that the first and last layers are both 256x256 just like our resized data.

In [ ]:
model.summary()

## Set weights for classes
Most of the lunar surface is in fact not an impact crater so we can set weights to make the background less important for the model.



In [13]:
def add_sample_weights(image, label):
    # The weights for each class, with the constraint that:
    #     sum(class_weights) == 1.0
    class_weights = tf.constant([0.05, 1]) # the first weight is for the background and the second for the craters.
    class_weights = class_weights/tf.reduce_sum(class_weights)

    # Create an image of `sample_weights` by using the label at each pixel as an 
    # index into the `class weights` .
    sample_weights = tf.gather(class_weights, indices=tf.cast(label, tf.int32))

    return image, label, sample_weights

## Train the U-net

In [ ]:
result = model.fit(train_batches.map(add_sample_weights), epochs=5, steps_per_epoch=100)

## Plot the training history

In [ ]:
# summarize history for accuracy
plt.plot(result.history['f1_m'])
plt.title('Model accuracy using f1 score')
plt.ylabel('f1 score')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


## Evaluate the model by applying it to the test images

In [ ]:
results = model.evaluate(test_batches)

This model is not the best in the world and some of that might be due to the fact that only large craters are labeled.

In [ ]:
# size of plot
plt.rcParams['figure.figsize'] = [16, 16]
f, axarr = plt.subplots(1,3)

#image 
testimage = X_test[0]
orgimage = np.squeeze(testimage)

# label
testlabel = Y_test[0]
testlabel = np.squeeze(testlabel)

# prediction
testimage = X_test[0]
testimage = np.reshape(testimage,[1,256,256,1])
prediction = model.predict(testimage)
pred = np.squeeze(prediction)


# Image
axarr[0].set_title('Original image')
axarr[0].imshow(orgimage, cmap='Greys_r')

# Label
axarr[1].set_title('Labeled craters')
axarr[1].imshow(testlabel)

# Prediction
axarr[2].set_title('Predicted craters')
axarr[2].imshow(pred>0.5)

## Plot predicted craters on top of the original image

In [ ]:
plt.rcParams['figure.figsize'] = [16, 16]
f, axarr = plt.subplots(1,2)

axarr[0].set_title('Original image and labels')
axarr[0].imshow(orgimage, cmap='Greys_r')
axarr[0].imshow(testlabel, alpha=0.7)

axarr[1].set_title('Original image and predictions')
axarr[1].imshow(orgimage, cmap='Greys_r')
axarr[1].imshow((pred>0.5), alpha=0.7)



## Questions



1.   Does including more images produce better results?
2.   Does an increased batch size improve the model?
3.   Does the model preform better when training for more epochs?
4.   How would the Unet be constructed if you wanted to train on the original larger images (512x512).
5.   How much faster is training on a GPU compared to a CPU? (tip: use the %%time command)





---



---


## Reading

https://arxiv.org/abs/1505.04597

https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d

https://naokishibuya.medium.com/up-sampling-with-transposed-convolution-9ae4f2df52d0


Big thanks to Florian Westphal for making this demo work.

## Contact
William.lidberg@slu.se

Florian.westphal@ju.se



